In [ ]:
from bokeh.plotting import output_notebook, figure, show
output_notebook()

import caret_analyze as caret
import caret_analyze.plot as caret_plot

In [ ]:
lttng = caret.Lttng('end_to_end_sample')
arch = caret.Architecture('yaml', 'architecture.yaml')
app = caret.Application(arch, lttng)
path = app.paths[0]

In [ ]:
caret_plot.message_flow(path, granularity='node')

In [ ]:
caret_plot.message_flow(path, granularity='raw')